In [1]:
import psycopg2

PORT = 54320

# connect to the postgresql db
conn = psycopg2.connect(
  host="localhost",
  port=PORT,
  dbname="coolseel",
  user="coolseel",
  password="password"
)

cur = conn.cursor()

In [51]:
# SELECT streamer.name, COUNT(stream) COUNT(segment), COUNT(message)
# FROM streamer
# LEFT JOIN stream ON streamer.id = stream.streamer_id
# LEFT JOIN segment ON stream.id = segment.stream_id
# LEFT JOIN message ON segment.id = message.segment_id
# GROUP BY streamer.name
# ORDER BY COUNT(stream) DESC

cur.execute("""
SELECT "Streamer".name, COUNT(distinct "Stream".id), COUNT(distinct "Segment".id), COUNT(distinct "Message".id)
FROM "Streamer"
LEFT JOIN "Stream" ON "Streamer".id = "Stream"."streamerId"
LEFT JOIN "Segment" ON "Stream".id = "Segment"."streamId"
LEFT JOIN "Message" ON "Segment".id = "Message"."segmentId"
WHERE "Stream".live = true
GROUP BY "Streamer".name
ORDER BY COUNT("Stream".id) DESC
""")
rows = cur.fetchall()
print("Streamer, Stream Count, Segment Count, Message Count")
for row in rows:
  print(row)

Streamer, Stream Count, Segment Count, Message Count
('northernlion', 2, 6, 113734)
('fanfan', 1, 1, 2608)
('boxbox', 1, 2, 1851)


In [46]:
conn.rollback()

In [52]:
# Select all segements associated with streamer 1
cur.execute("""
SELECT "Segment".id, "Stream".id, "Stream".live, "Segment".start, "Segment".game
FROM "Segment"
LEFT JOIN "Stream" ON "Segment"."streamId" = "Stream".id
WHERE "Stream"."streamerId" = 1
""")
rows = cur.fetchall()
print("Segment ID, Stream ID, Start, Game")
for row in rows:
  print(row)

Segment ID, Stream ID, Start, Game
(149, 114, True, datetime.datetime(2023, 12, 15, 17, 17, 7, 151000), 'Against the Storm')
(150, 114, True, datetime.datetime(2023, 12, 15, 20, 1, 38, 67000), 'Lethal Company')
(151, 115, True, datetime.datetime(2023, 12, 14, 17, 14, 9, 394000), 'Super Auto Pets')
(152, 115, True, datetime.datetime(2023, 12, 14, 17, 18, 49, 414000), 'House Flipper 2')
(153, 115, True, datetime.datetime(2023, 12, 14, 19, 4, 29, 855000), 'Jackbox Party Packs')
(154, 115, True, datetime.datetime(2023, 12, 14, 20, 37, 30, 175000), 'House Flipper 2')
(155, 116, False, datetime.datetime(2023, 12, 16, 18, 47, 2, 248000), 'OFFLINE')
(156, 117, False, datetime.datetime(2023, 12, 15, 9, 47, 9, 192000), 'OFFLINE')
(157, 118, False, datetime.datetime(2023, 12, 17, 10, 17, 39, 524000), 'OFFLINE')
(158, 119, False, datetime.datetime(2023, 12, 15, 22, 29, 58, 815000), 'OFFLINE')
(159, 120, False, datetime.datetime(2023, 12, 15, 21, 59, 48, 678000), 'OFFLINE')


In [6]:
cur.execute("""
SELECT "Stream"."twitchId", "Streamer".name
FROM "Stream"
LEFT JOIN "Streamer" ON "Stream"."streamerId" = "Streamer".id
""")
rows = cur.fetchall()
print("Stream ID, Streamer Name")
for row in rows:
  print(row)

Stream ID, Streamer Name
('43242889451', 'boxbox')
('43247623627', 'fanfan')
('2004847346', 'northernlion')
('2003978864', 'northernlion')
(None, 'northernlion')
(None, 'northernlion')
(None, 'northernlion')
(None, 'northernlion')
(None, 'northernlion')
('PRESTREAM_post', 'northernlion')


In [34]:
# insert 3 fake streams with 100000 messages each for streamer 1
for i in range(3):
  cur.execute("""
  INSERT INTO "Stream" ("streamerId", "live", "start")
  VALUES (1, true, NOW())
  RETURNING id
  """)
  stream_id = cur.fetchone()[0]
  cur.execute("""
  INSERT INTO "Segment" ("streamId", game, start)
  VALUES (%s, %s, NOW())
  RETURNING id
  """, (stream_id, "Test Game"))
  segment_id = cur.fetchone()[0]
  for j in range(100000):
    cur.execute("""
    INSERT INTO "Message" ("segmentId", "secondsSinceStart", "text", "username")
    VALUES (%s, %s, %s, %s)
    """, (segment_id, j, f"This is a test message number {j}", "test_user"))

In [37]:
# delete all messages from "test_user"
cur.execute("""
DELETE FROM "Message"
WHERE "username" = 'test_user'
""")
# delete all segments with no messages
cur.execute("""
DELETE FROM "Segment"
WHERE NOT EXISTS (
  SELECT 1
  FROM "Message"
  WHERE "Message"."segmentId" = "Segment".id
)
""")
# delete all streams with no segments
cur.execute("""
DELETE FROM "Stream"
WHERE NOT EXISTS (
  SELECT 1
  FROM "Segment"
  WHERE "Segment"."streamId" = "Stream".id
)
""")

In [38]:
conn.commit()

In [5]:
conn.rollback()